### Data preparation
This notebook is for the purpose of preparaing images collected and set them up in proper format for training with YOLO algorithm

In [2]:
from pathlib import Path
import os
import shutil
import random
import glob

In [99]:
def get_all_images_paths(folder_path):
    
        image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.gif"]  # Add more extensions if needed
        image_paths = []
        for extension in image_extensions:
            image_paths.extend(glob.glob(os.path.join(folder_path, extension)))
        print(f"There are {len(image_paths)} images found ")
        return image_paths
images_path_train_open = get_all_images_paths(r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\Datasets\cleaned_images\Training\Open" )
images_path_train_close = get_all_images_paths(r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\Datasets\cleaned_images\Training\close_2")
images_path_test_open =  get_all_images_paths(r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\Datasets\cleaned_images\Validation\Open")
images_path_test_close = get_all_images_paths(r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\Datasets\cleaned_images\Validation\Close_2")

There are 269 images found 
There are 229 images found 
There are 40 images found 
There are 40 images found 


In [93]:
#rename images in each folder
def rename_images(path):
    for root,dirs,files in os.walk(path):
        for filename in files:
            if filename.endswith((".jpg",".png",".jpeg")):
                parent_folder = os.path.basename(root)
                grandparentfolder = os.path.basename(os.path.dirname(root))
                old_path = os.path.join(root,filename)
                new_filename = f"{grandparentfolder}_{parent_folder}_{filename}"
                new_path = os.path.join(root,new_filename)
                os.rename(old_path,new_path)
rename_images(r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\Datasets\cleaned_images\Training\Open")
rename_images(r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\Datasets\cleaned_images\Training\close_2")
rename_images(r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\Datasets\cleaned_images\Validation\Open")
rename_images(r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\Datasets\cleaned_images\Validation\Close_2")

In [100]:
random.seed(42)

#select and remove 90 images for validation from 
all_images_path_val_open = random.sample(list(images_path_train_open),50)
all_images_path_val_close = random.sample(list(images_path_train_close),29)

#remmove validation data from training
all_images_path_train_open = [img_path for img_path in images_path_train_open if img_path not in all_images_path_val_open]
all_images_path_train_close =  [img_path for img_path in images_path_train_close if img_path not in all_images_path_val_close]
print(f"Current number of training images after picking some samples for validation is {len(list(all_images_path_train_open))} for opened eyes")
print(f"Current number of training images after picking some samples for validation is {len(list(all_images_path_train_close))} for closed eyes")

Current number of training images after picking some samples for validation is 219 for opened eyes
Current number of training images after picking some samples for validation is 200 for closed eyes


In [101]:
os.chdir(r"C:\Users\User\Desktop\CV\Driver_dizziness_detector")

#make direcorty for test,train and validation data
images_path_dirs = [data_path / "test",data_path / "train", data_path / "valid"]
for dir_name in images_path_dirs:
  if dir_name.is_dir():
       print(f"Directory {dir_name} already exists")
  else:
       os.mkdir(dir_name)

In [103]:
def copy_images(image_paths,destination_folder):
    for image_path in image_paths:
        image_name = image_path.split("\\")[-1]  # Extract the image file name
        destination_path = destination_folder + "\\"  +  image_name  # Create the destination path
        # Copy the image file to the destination folder
       
        shutil.copy(image_path, destination_path)
            

In [104]:
copy_images(image_paths = all_images_path_train_open,destination_folder = r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\train") #copy respective train images to required folder
copy_images(image_paths = all_images_path_train_close,destination_folder = r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\train") #copy respective train images to required folder
copy_images(image_paths = images_path_test_close,destination_folder = r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\test") # copy respective test images to required folder
copy_images(image_paths = images_path_test_open,destination_folder = r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\test")
copy_images(image_paths = all_images_path_val_close,destination_folder = r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\valid") 
copy_images(image_paths = all_images_path_val_open,destination_folder = r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\valid")  ##copy respective validation images images to required folder

### Moving all labels text files to separate folders

In [14]:
#Separate training images and labelled text files in seprate folders
train_path_to_files = r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\train"
test_path_to_files =  r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\test"
valid_path_to_files =  r"C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\valid"
def get_all_images_txt_paths(folder_path):
        image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.gif"]  # Add more extensions if needed
        image_paths = []
        for extension in image_extensions:
            image_paths.extend(glob.glob(os.path.join(folder_path, extension)))
        print(f"There are {len(image_paths)} images found in {folder_path}")

        txt_paths = []
        txt_extension = "*.txt"
        txt_paths.extend(glob.glob(os.path.join(folder_path, txt_extension)))
        print(f"There are {len(txt_paths)} txt files  found in {folder_path}")

        #create new folders form imgages and labels
        #chnage dir first
        os.chdir(folder_path)
        folders_to_create  = ["images/","labels/"]
        for folder in folders_to_create:
            if os.path.isdir(folder):
                print(f"{folder} already exists")
            else:
                os.mkdir(folder)
        #copy imgages to to images folder
        for img_path in image_paths:
            image_name = img_path.split("\\")[-1]
            destination_path = folder_path +  "\\" + "images" + "\\" + image_name
            shutil.copy(img_path, destination_path)
        #copy all test files to labels folder
        for txt_path in txt_paths:
            txt_name = txt_path.split("\\")[-1]
            destination_path = folder_path +  "\\" + "labels" + "\\" + txt_name
            shutil.copy(txt_path, destination_path)
        

get_all_images_txt_paths(valid_path_to_files) #copy imgaes and labels for validation data
get_all_images_txt_paths(test_path_to_files) #copy images and labels for test data
get_all_images_txt_paths(train_path_to_files) #copy images and labels for train data

There are 79 images found in C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\valid
There are 79 txt files  found in C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\valid
There are 71 images found in C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\test
There are 71 txt files  found in C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\test
There are 379 images found in C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\train
There are 379 txt files  found in C:\Users\User\Desktop\CV\Driver_dizziness_detector\prepared_data\train
